# Bet Against Beta Portfolio #

In [1]:
# Import Libraries

# Data Management
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from portfolios_toolkit import calculate_analytics
from capm_toolkit import capm_regression

In [2]:
# Import Data
df_returns = pd.read_csv(r'..\additional_data\stocks_returns.csv')
df_returns = df_returns.rename(columns={'Unnamed: 0':'Date'})
df_returns.set_index('Date', inplace=True)
df_returns.index = pd.to_datetime(df_returns.index)
df_returns = df_returns.dropna(axis=1)

df_returns

In [3]:
# Get the important data for the Risk-Free Rate
rfr = pd.read_csv(r"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index)
rfr.dropna(inplace = True)

# Get the important data for the S&P500
data_sp500 = pd.read_csv(r'..\additional_data\sp500.csv')
data_sp500.set_index('Date', inplace=True)
data_sp500.index = pd.to_datetime(data_sp500.index)
data_sp500 = data_sp500['sp_500']

### Calculate Betas ###

In [4]:
# Calculate the Market Excess Returns
market_excess_returns = (data_sp500 - rfr['risk_free_rate']).dropna()
market_excess_returns.name = 'market_excess_returns'

market_excess_returns

In [5]:
# Calculate Stocks Excess Returns
df_excess_returns = df_returns.sub(rfr['risk_free_rate'], axis=0)
df_excess_returns.dropna(inplace = True)

df_excess_returns

In [7]:
# Set the Window
window = len(df_excess_returns.loc['2015':'2019'])

# Create the Betas and Alpha + Residuals DataFrames for the whole time stamp
betas_dict = {}
capm_betas_dict = {}

# Loop to Obtain Betas and Alpha + Residuals
for ticker in df_excess_returns.columns:
    
    # Fit the WLS model
    model = capm_regression(
        df_excess_returns[ticker].loc['2015':'2019'], 
        market_excess_returns.loc['2015':'2019'],
        window=window,
        WLS=True
    )

    # Extract Alpha and Beta
    alpha = model.params.iloc[0]
    beta = model.params.iloc[1]

    # Store Beta
    betas_dict[ticker] = beta

# Create Beta Series
betas_series = pd.Series(betas_dict)
betas_series.name = 'beta'

betas_series.sort_values(ascending=False)

### First Strategy: Long Only Betas ###

In [8]:
long_beta_portfolio_weights = betas_series / betas_series.sum()
long_beta_portfolio_weights.name = 'weights'

long_beta_portfolio_weights

In [9]:
# Portfolio Returns
long_portfolio_returns = df_returns.loc['2020':] @ long_beta_portfolio_weights
long_portfolio_returns.name = 'long_portfolio_returns'

long_portfolio_returns

In [10]:
long_portfolio_returns.mean()

In [50]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(long_portfolio_returns.cumsum().mul(100), label='Long-Only Portfolio', color='black', alpha=1)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns (%)')
plt.legend()
plt.grid()

# Show
plt.show() 

### Second Strategy: Betting Agains Beta ###

In [12]:
# Shrinking Betas
shrunk_betas = 0.6 * betas_series + 0.4

In [13]:
shrunk_betas.sort_values(ascending=False)

In [51]:
# Fist, calculate Ranks
ranks = shrunk_betas.rank()

ranks

In [52]:
# Median
median_rank = ranks.median()

median_rank

In [53]:
# Second: define the groups
low_beta = ranks[ranks < median_rank]
high_beta = ranks[ranks >= median_rank]

In [16]:
# Non-scaled weights
z_bar = ranks.mean()

w_low = (z_bar - low_beta).clip(lower=0)
w_low = w_low / w_low.sum()

w_high = (high_beta - z_bar).clip(lower=0)
w_high = w_high / w_high.sum()

In [17]:
w_low

In [18]:
w_high

In [54]:
# Scale to make beta neutral
beta_low = (w_low * shrunk_betas[w_low.index]).sum()
beta_high = (w_high * shrunk_betas[w_high.index]).sum()

w_low_scaled = w_low / beta_low
w_high_scaled = w_high / beta_high

In [55]:
bab_portfolio_weights = pd.concat([w_low_scaled, -w_high_scaled])
bab_portfolio_weights.name = 'weights'

bab_portfolio_weights.sort_values(ascending=True)

In [56]:
bab_portfolio_weights.sum().round(3)

In [57]:
# Portfolio Returns
bab_portfolio_returns = df_returns.loc['2020':] @ bab_portfolio_weights
bab_portfolio_returns.name = 'bab_portfolio_returns'

bab_portfolio_returns

In [23]:
bab_portfolio_returns.mean()

In [58]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(bab_portfolio_returns.cumsum().mul(100), label='Betting-Against-Beta Portfolio', color='black', alpha=1)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns (%)')
plt.legend()
plt.grid()

# Show
plt.show() 

### Compare the Strategies ###

In [59]:
# Create DataFrame
strategies_df = pd.DataFrame(index = df_returns.loc['2020':].index)
strategies_df.index.name = 'date'
strategies_df['long_portfolio'] = long_portfolio_returns
strategies_df['bab_portfolio'] = bab_portfolio_returns

strategies_df

In [61]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(strategies_df.cumsum().mul(100), label=strategies_df.columns, alpha=1)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns (%)')
plt.legend()
plt.grid()

# Show
plt.show() 

In [62]:
# Analytics
analytics = calculate_analytics(strategies_df)

analytics

### Calculate the Portfolio Beta ###

In [63]:
# Create a DataFrame
regression_df = pd.DataFrame()
regression_df['portfolio_excess'] = bab_portfolio_returns - rfr['risk_free_rate'].loc['2020':]
regression_df['market_excess'] = market_excess_returns.loc['2020':]
regression_df.dropna(inplace = True)

regression_df

In [64]:
# Fit the WLS model
portfolio_model = capm_regression(
    regression_df['portfolio_excess'], 
    regression_df['market_excess'],
    window=len(regression_df),
    WLS=True
)

print(portfolio_model.summary())

### Implementing Rebalancing ###

In [42]:
# Function to calculate the weights (as we saw above)
def betting_against_beta_weights(
    betas: pd.Series
) -> pd.Series:
    
    # Drop NANs
    betas = betas.dropna()

    # Shrinkage
    w_shrink = 0.6      # Adjust
    betas_shrunk = w_shrink * betas + (1 - w_shrink) * 1.0

    # Ranking Betas
    ranks = betas_shrunk.rank()
    z_bar = ranks.mean()
    median_rank = ranks.median()

    # Split using the Median
    low_beta = ranks[ranks < median_rank]
    high_beta = ranks[ranks >= median_rank]

    # Calculate Weights
    w_low = (z_bar - low_beta).clip(lower=0)
    w_high = (high_beta - z_bar).clip(lower=0)

    # Standardize Weights
    w_low /= w_low.sum()
    w_high /= w_high.sum()

    # Scale betas so each side has a beta close to 1
    beta_low = (w_low * betas_shrunk[w_low.index]).sum()
    beta_high = (w_high * betas_shrunk[w_high.index]).sum()

    # Standardize Again
    w_low_scaled = w_low / beta_low
    w_high_scaled = w_high / beta_high

    # Concat
    bab_weights = pd.concat([w_low_scaled, -w_high_scaled])

    return bab_weights

In [43]:
# Fortunately, we calculated the betas (with a 252-d history in the previous notebook)
rolling_betas = pd.read_csv(r'..\additional_data\capm_hbetas.csv')
rolling_betas.set_index('date', inplace=True)
rolling_betas.index = pd.to_datetime(rolling_betas.index)
rolling_betas = rolling_betas[df_returns.columns]

rolling_betas

In [65]:
# Function for the Rolling Weights
def calculate_bab_rolling_weights(
    beta_df, 
    rebalance_days=21
):
    # Rebalancing Dates
    rebalance_dates = beta_df.index[::rebalance_days]

    # List used for storing
    weights_list = []

    # Loop
    for date in rebalance_dates:
        # Betas for each date
        betas_today = beta_df.loc[date]
        
        # Calculate Weights and store them
        weights = betting_against_beta_weights(betas_today)
        weights.name = date
        weights_list.append(weights)

    # Create a DataFrame
    bab_weights_rebalance = pd.DataFrame(weights_list)

    # Reindexing for daily weights
    bab_weights_daily = bab_weights_rebalance.reindex(beta_df.index)

    # Forward Fill
    bab_weights_daily = bab_weights_daily.ffill().fillna(0)

    # Reindexing Columns to have consistency
    bab_weights_daily = bab_weights_daily.reindex(columns=beta_df.columns).fillna(0)

    return bab_weights_daily


In [66]:
# Get the Weights
bab_daily_weights = calculate_bab_rolling_weights(rolling_betas, rebalance_days=21)

bab_daily_weights

In [69]:
# Calculate the Portfolio using rebalancing
bab_rebalancing_portfolio = (df_returns.loc[bab_daily_weights.index] * bab_daily_weights).sum(axis = 1)
bab_rebalancing_portfolio.name = 'bab_rebalancing_portfolio'

bab_rebalancing_portfolio

In [71]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(bab_rebalancing_portfolio.cumsum().mul(100), label='BAB Strategy with Rebalacing', alpha=1)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns (%)')
plt.legend()
plt.grid()

# Show
plt.show() 

In [73]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(strategies_df.cumsum().mul(100), label=strategies_df.columns, alpha=1)
plt.plot(bab_rebalancing_portfolio.loc['2020':].cumsum().mul(100), label='BAB Strategy with Rebalacing', alpha=1)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns (%)')
plt.legend()
plt.grid()

# Show
plt.show() 